# Classification 3. Hierarchical-Separate Approach 

In [4]:
!pip install transformers

In [1]:
from transformers import BertTokenizer
import torch
import numpy as np
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv("beer_df_large.csv", sep="\t")

In [4]:
df = df.drop(columns=['Name', 'Company', 'Region', 'ABV', 'Avg', 'Rate'])

In [5]:
df

,Group,Style,Review
0,Bocks,Bock,From a 12oz bottle into a cocktail glass.\n\nC...
1,Bocks,Bock,"Great relaxing beer. Very mellow, great taste-..."
2,Bocks,Bock,"Appearance: Clear, bright copper color. Frothy..."
3,Bocks,Bock,As a German staying for holidays in the US I c...
4,Bocks,Bock,I would guess this is Shiner's #1 beer. It's p...
...,...,...,...
174679,Wild/Sour Beers,Wild Ale,"A really well done, well balanced sour with an..."
174680,Wild/Sour Beers,Wild Ale,"22oz bottle. Poured out a slightly hazy, brigh..."
174681,Wild/Sour Beers,Wild Ale,"There's lots of apricot in this beer, just a t..."
174682,Wild/Sour Beers,Wild Ale,Poured from a 22oz bomber into my Drie de Font...


In [6]:
groups = df['Group'].value_counts().index.tolist()
groups

['Pale Lagers',
 'Pale Ales',
 'Strong Ales',
 'Stouts',
 'India Pale Ales',
 'Wild/Sour Beers',
 'Specialty Beers',
 'Dark Lagers',
 'Porters',
 'Wheat Beers',
 'Brown Ales',
 'Bocks',
 'Dark Ales']

In [7]:
group_dfs = []
for g in groups:
    new_df = df[df['Group'] == g]
    new_df = new_df.drop('Group', axis=1)
    group_dfs.append(new_df)

In [8]:
group_dfs[0]

,Style,Review
66877,American Adjunct Lager,"Sweet, clean, green apples, mild bitterness. S..."
66878,American Adjunct Lager,Look...Clear pale yellow with a white carbonat...
66879,American Adjunct Lager,Look: Crystal clear yellow with an awesome-loo...
66880,American Adjunct Lager,Defies comparison. Hating on this is like comp...
66881,American Adjunct Lager,"Might be my least favorite AAL, mostly from th..."
...,...,...
93688,Malt Liquor,"Bottled in the 1970's era, and drank almost 50..."
93689,Malt Liquor,"Small pour at brewery. Looks lika glass of OJ,..."
93690,Malt Liquor,Small pour at brewery. Copper body with tiny h...
93691,Malt Liquor,"Four pack from brewery. Dark golden, patchy he..."


In [12]:
#df['Review'] = df.apply(lambda x: 'Group: {}. Review: {}'.format(x['Group'], x['Review']), axis=1)
#df = df.drop('Group', axis=1)

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [10]:
lables_list = []
for gd in group_dfs:
    labels = {}
    for i, name in enumerate(gd['Style'].value_counts().index.tolist()):
        labels[name] = i
    lables_list.append(labels)

In [19]:
lables_list[2]

{'Tripel': 0,
 'Belgian Pale Strong Ale': 1,
 'Belgian Dark Strong Ale': 2,
 'English Barleywine': 3,
 'American Barleywine': 4,
 'Scotch Ale / Wee Heavy': 5,
 'Quadrupel (Quad)': 6,
 'American Strong Ale': 7,
 'Old Ale': 8,
 'Imperial Red Ale': 9,
 'English Strong Ale': 10,
 'Wheatwine': 11}

In [26]:
res = {}

In [27]:
for i, name in enumerate(groups):
    res[name] = {id: style for style, id in lables_list[i].items()}

In [28]:
res

{'Pale Lagers': {0: 'American Adjunct Lager',
  1: 'German Pilsner',
  2: 'European Pale Lager',
  3: 'American Lager',
  4: 'Helles',
  5: 'Bohemian / Czech Pilsner',
  6: 'Light Lager',
  7: 'Kellerbier / Zwickelbier',
  8: 'European / Dortmunder Export Lager',
  9: 'Festbier / Wiesnbier',
  10: 'European Strong Lager',
  11: 'Malt Liquor',
  12: 'India Pale Lager (IPL)',
  13: 'Imperial Pilsner',
  14: 'Czech Pale Lager'},
 'Pale Ales': {0: 'English Pale Ale',
  1: 'American Blonde Ale',
  2: 'Saison',
  3: 'American Pale Ale',
  4: 'Kölsch',
  5: 'American Amber / Red Ale',
  6: 'Belgian Pale Ale',
  7: 'English Bitter',
  8: 'Extra Special / Strong Bitter (ESB)',
  9: 'Irish Red Ale',
  10: 'Belgian Blonde Ale',
  11: 'Bière de Garde',
  12: 'Grisette',
  13: 'English Pale Mild Ale'},
 'Strong Ales': {0: 'Tripel',
  1: 'Belgian Pale Strong Ale',
  2: 'Belgian Dark Strong Ale',
  3: 'English Barleywine',
  4: 'American Barleywine',
  5: 'Scotch Ale / Wee Heavy',
  6: 'Quadrupel (Qu

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df_, labels_):
        self.labels = [labels_[label] for label in df_['Style']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True, 
                                return_tensors="pt") 
                      for text in df_['Review']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [18]:
!pip install sklearn


  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=855bf7a0d608464fd6665585671b00ce0ad48d3a3599726c7fd197b273976930
  Stored in directory: /root/.cache/pip/wheels/15/9a/b1/2478e73a520d596fab614693f5cd1beef4ba3db737bed1ac7d
Successfully built sklearn


In [13]:
from sklearn.model_selection import train_test_split

df_splits = []
for gd in group_dfs:
    df_train, df_test = train_test_split(gd, train_size = 0.9, random_state=42, stratify = gd['Style'])
    df_train, df_val = train_test_split(df_train, train_size = 0.88889, random_state=42, stratify = df_train['Style'])
    df_splits.append((df_train, df_val, df_test))
    print(len(df_train), len(df_val), len(df_test))

21452 2682 2682
18504 2313 2313
16604 2076 2076
13391 1674 1674
11332 1417 1417
10658 1333 1333
9759 1220 1220
9737 1218 1218
7602 951 951
6775 847 847
5589 699 699
4652 582 582
3683 461 461


In [14]:
class BertClassifier(nn.Module):
    def __init__(self, num_classes, dropout=0.2):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [15]:
def train(model, train_data, val_data, learning_rate, epochs, labels):
    print('Loading data into a dataset...')

    train, val = Dataset(train_data, labels), Dataset(val_data, labels)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
    print('Data loaded')

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [17]:
EPOCHS = 5
models = []
LR = 1e-6

for ds in range(len(df_splits)):
    model = BertClassifier(len(lables_list[ds]))
    print(f'Training for group: {groups[ds]}')
    train(model, df_splits[ds][0], df_splits[ds][1], LR, EPOCHS, lables_list[ds])
    torch.save(model, f'{ds}.pt')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training for group: Pale Lagers
Loading data into a dataset...
Data loaded


100%|██████████| 10726/10726 [11:20<00:00, 15.75it/s]


Epochs: 1 | Train Loss:  1.212                 | Train Accuracy:  0.210                 | Val Loss:  1.054                 | Val Accuracy:  0.340


100%|██████████| 10726/10726 [11:37<00:00, 15.38it/s]


...

Epochs: 3 | Train Loss:  0.257                 | Train Accuracy:  0.814                 | Val Loss:  0.304                 | Val Accuracy:  0.755


100%|██████████| 1842/1842 [01:56<00:00, 15.75it/s]


Epochs: 4 | Train Loss:  0.194                 | Train Accuracy:  0.865                 | Val Loss:  0.292                 | Val Accuracy:  0.779


100%|██████████| 1842/1842 [01:56<00:00, 15.81it/s]


Epochs: 5 | Train Loss:  0.142                 | Train Accuracy:  0.920                 | Val Loss:  0.293                 | Val Accuracy:  0.807


In [ ]:
def evaluate(model, test_data):
    y_pred = []
    y_true = []
    print('Loading data...')
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    print('Data loaded')

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    
    with torch.no_grad():
        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            y_pred.append(output.argmax(dim=1))
            y_true.append(test_label)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return y_pred, y_true
    

In [ ]:
for m in range(len(models)):
    y_pred, y_true = evaluate(models[m], df_splits[2])

In [ ]:
import torch
load = torch.load('beert_model_simple.pt')

confusion matrix

In [ ]:
t = [x.cpu() for x in y_true]

In [ ]:
classes = labels.keys()
cf_matrix = confusion_matrix([x.cpu() for x in y_true], [y.cpu() for y in y_pred])
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')